In [33]:
from datetime import datetime
from importlib import reload
from vnpy.app.portfolio_strategy import BacktestingEngine
from vnpy.trader.constant import Interval


# ARKW
symbols_ARKW = ['ADBE', 'ADYEY', 'API', 'BABA', 'BEKE', 'CRM', 'CRWD', 'DOCU', 'FB', 'FSLY', 'GBTC', 'HUBS', 'HUYA', 'ICE', 
           'IPOB', 'JD', 'LC', 'MELI', 'NET', 'NFLX', 'NTDOY', 'NVDA', 'OKTA', 'PD', 'PINS', 'PLTR', 'PSTG', 'PTON', 'PYPL',
           'ROKU', 'SE', 'SHOP', 'SNAP', 'SNPS', 'SPLK', 'SPOT', 'SQ', 'TCEHY', 'TDOC', 'TEAM', 'TREE', 'TSLA', 'TSM', 'TTD', 'TWLO', 
           'TWOU', 'U', 'VCYT', 'WORK', 'Z', 'ZM']

# ARKK
symbols_ARKK = ['BEKE', 'CERS', 'CGEN', 'CRSP', 'DOCU', 'DOYU', 'EDIT', 'EXAS', 'HUYA', 'ICE', 'IOVA', 'IRDM', 'MCRB', 'MTLS', 'NSTG',
           'NTDOY', 'NTLA', 'NVTA', 'ONVO', 'PACB', 'PD', 'PINS', 'PRLB', 'PSTG', 'PYPL', 'ROKU', 'SE', 'SHOP', 'SNPS', 'SPLK', 
           'SPOT', 'SQ', 'SSYS', 'SYRS', 'TCEHY', 'TDOC', 'TREE', 'TSLA', 'TSM', 'TWLO', 'TWOU', 'TWST', 'U', 'VCYT', 'WORK', 'XONE',
           'Z', 'ZM']

# ARKQ
symbols_ARKQ = ['AAPL', 'ADSK', 'AMZN', 'ANSS', 'AVAV', 'BIDU', 'BYDDY', 'CAT', 'DE', 'ESLT', 'FLIR', 'GOOG', 'HON', 'IRDM', 'ISRG',
           'JD', 'KMTUY', 'KTOS', 'MTLS', 'NIU', 'NNDM', 'NVDA', 'NXPI', 'ONVO', 'PRLB', 'RAVN', 'ROK', 'SNPS', 'SPCE', 'SPLK', 
           'SSYS', 'TCEHY', 'TER', 'TRMB', 'TSLA', 'TSM', 'TWOU', 'WKHS', 'XLNX', 'XONE']



In [34]:
def generate_vt_symbols(symbols):
    """
    合成回测用的vt_symbol列表
    vt_symbol格式为：合约代码.交易所（FUBO-USD-STK.SMART）
    """
    vt_symbols = []
    for i in symbols:
        symbol = i + '-USD-STK' + '.SMART'
        vt_symbols.append(symbol)
    return vt_symbols


In [35]:
def set_ratios(num, vt_symbols):
    """
    设置每个vt_symbol的回测参数，最终返回字典格式
    用于设置手续费，滑点，合约规模，price tick
    """
    d = {}
    for i in vt_symbols:
        d[i] = num
    return d

In [36]:
def show_resullt(
    vt_symbols,
    strategy,
    setting={},
    show_chart=True
):
    # 1)创建回测引擎
    engine = BacktestingEngine()
    
    # 2）设置回测参数
    engine.set_parameters(
        vt_symbols=vt_symbols,
        interval=Interval.DAILY,
        start=datetime(2018, 12,10),
        end=datetime(2020, 12, 8),
        rates=set_ratios(1/10000, vt_symbols),
        slippages=set_ratios(0, vt_symbols),
        sizes=set_ratios(1, vt_symbols),
        priceticks=set_ratios(0.01, vt_symbols),
        capital=5_000_000,
    )
    
    # 3)添加策略
    engine.add_strategy(strategy, setting)
    # 4) 加载历史数据
    engine.load_data()
    # 5) 跑回测，基于事件引擎逐条回放，得到成交记录
    engine.run_backtesting()
    # 6) 基于逐日顶市规则和成交记录，得到组合资金曲线
    df = engine.calculate_result()
    # 7) 基于资金曲线，计算策略回测指标，如收益率，夏普比率
    engine.calculate_statistics()
    # 8) 画图
    if show_chart:
        engine.show_chart()

In [37]:
# 生成可交易的合约
all_vt_symbols = generate_vt_symbols(symbols_ARKK)
print(all_vt_symbols)

['BEKE-USD-STK.SMART', 'CERS-USD-STK.SMART', 'CGEN-USD-STK.SMART', 'CRSP-USD-STK.SMART', 'DOCU-USD-STK.SMART', 'DOYU-USD-STK.SMART', 'EDIT-USD-STK.SMART', 'EXAS-USD-STK.SMART', 'HUYA-USD-STK.SMART', 'ICE-USD-STK.SMART', 'IOVA-USD-STK.SMART', 'IRDM-USD-STK.SMART', 'MCRB-USD-STK.SMART', 'MTLS-USD-STK.SMART', 'NSTG-USD-STK.SMART', 'NTDOY-USD-STK.SMART', 'NTLA-USD-STK.SMART', 'NVTA-USD-STK.SMART', 'ONVO-USD-STK.SMART', 'PACB-USD-STK.SMART', 'PD-USD-STK.SMART', 'PINS-USD-STK.SMART', 'PRLB-USD-STK.SMART', 'PSTG-USD-STK.SMART', 'PYPL-USD-STK.SMART', 'ROKU-USD-STK.SMART', 'SE-USD-STK.SMART', 'SHOP-USD-STK.SMART', 'SNPS-USD-STK.SMART', 'SPLK-USD-STK.SMART', 'SPOT-USD-STK.SMART', 'SQ-USD-STK.SMART', 'SSYS-USD-STK.SMART', 'SYRS-USD-STK.SMART', 'TCEHY-USD-STK.SMART', 'TDOC-USD-STK.SMART', 'TREE-USD-STK.SMART', 'TSLA-USD-STK.SMART', 'TSM-USD-STK.SMART', 'TWLO-USD-STK.SMART', 'TWOU-USD-STK.SMART', 'TWST-USD-STK.SMART', 'U-USD-STK.SMART', 'VCYT-USD-STK.SMART', 'WORK-USD-STK.SMART', 'XONE-USD-STK.SMAR

In [38]:
#加载策略。一键回测
from vnpy.app.portfolio_strategy.strategies import trend_following_strategy
from vnpy.app.portfolio_strategy.strategies import pair_trading_strategy
from vnpy.app.portfolio_strategy.strategies import return10_strategy
from vnpy.app.portfolio_strategy.strategies import pcp_arbitrage_strategy

# show_resullt(all_vt_symbols, trend_following_strategy.TrendFollowingStrategy)
# show_resullt(all_vt_symbols, pair_trading_strategy.PairTradingStrategy)
reload(return10_strategy)
show_resullt(all_vt_symbols, return10_strategy.Return10Strategy)
# show_resullt(all_vt_symbols, pcp_arbitrage_strategy.PcpArbitrageStrategy)

2021-02-28 16:58:55.587883	开始加载历史数据
2021-02-28 16:58:55.589880	BEKE-USD-STK.SMART加载进度： [4%]
2021-02-28 16:58:55.590876	BEKE-USD-STK.SMART加载进度： [8%]
2021-02-28 16:58:55.591872	BEKE-USD-STK.SMART加载进度：# [12%]
2021-02-28 16:58:55.592870	BEKE-USD-STK.SMART加载进度：# [16%]
2021-02-28 16:58:55.593867	BEKE-USD-STK.SMART加载进度：## [21%]
2021-02-28 16:58:55.595863	BEKE-USD-STK.SMART加载进度：## [25%]
2021-02-28 16:58:55.596861	BEKE-USD-STK.SMART加载进度：## [29%]
2021-02-28 16:58:55.597859	BEKE-USD-STK.SMART加载进度：### [33%]
2021-02-28 16:58:55.598854	BEKE-USD-STK.SMART加载进度：### [37%]
2021-02-28 16:58:55.599851	BEKE-USD-STK.SMART加载进度：#### [41%]
2021-02-28 16:58:55.600849	BEKE-USD-STK.SMART加载进度：#### [45%]
2021-02-28 16:58:55.600849	BEKE-USD-STK.SMART加载进度：#### [49%]
2021-02-28 16:58:55.601846	BEKE-USD-STK.SMART加载进度：##### [53%]
2021-02-28 16:58:55.602843	BEKE-USD-STK.SMART加载进度：##### [58%]
2021-02-28 16:58:55.602843	BEKE-USD-STK.SMART加载进度：###### [62%]
2021-02-28 16:58:55.603840	BEKE-USD-STK.SMART加载进度：###### [66%]
2021-0

2021-02-28 16:58:55.951929	DOYU-USD-STK.SMART加载进度：######### [91%]
2021-02-28 16:58:55.957912	DOYU-USD-STK.SMART加载进度：######### [95%]
2021-02-28 16:58:55.959906	DOYU-USD-STK.SMART加载进度：######### [99%]
2021-02-28 16:58:55.959906	DOYU-USD-STK.SMART历史数据加载完成，数据量：354
2021-02-28 16:58:55.962899	EDIT-USD-STK.SMART加载进度： [4%]
2021-02-28 16:58:55.965891	EDIT-USD-STK.SMART加载进度： [8%]
2021-02-28 16:58:55.967885	EDIT-USD-STK.SMART加载进度：# [12%]
2021-02-28 16:58:55.970878	EDIT-USD-STK.SMART加载进度：# [16%]
2021-02-28 16:58:55.973869	EDIT-USD-STK.SMART加载进度：## [21%]
2021-02-28 16:58:55.976861	EDIT-USD-STK.SMART加载进度：## [25%]
2021-02-28 16:58:55.979854	EDIT-USD-STK.SMART加载进度：## [29%]
2021-02-28 16:58:55.983844	EDIT-USD-STK.SMART加载进度：### [33%]
2021-02-28 16:58:55.988831	EDIT-USD-STK.SMART加载进度：### [37%]
2021-02-28 16:58:55.991821	EDIT-USD-STK.SMART加载进度：#### [41%]
2021-02-28 16:58:55.995811	EDIT-USD-STK.SMART加载进度：#### [45%]
2021-02-28 16:58:55.997805	EDIT-USD-STK.SMART加载进度：#### [49%]
2021-02-28 16:58:56.000798	EDIT-

2021-02-28 16:58:56.513291	MTLS-USD-STK.SMART加载进度：#### [41%]
2021-02-28 16:58:56.515287	MTLS-USD-STK.SMART加载进度：#### [45%]
2021-02-28 16:58:56.518277	MTLS-USD-STK.SMART加载进度：#### [49%]
2021-02-28 16:58:56.520272	MTLS-USD-STK.SMART加载进度：##### [53%]
2021-02-28 16:58:56.524263	MTLS-USD-STK.SMART加载进度：##### [58%]
2021-02-28 16:58:56.527254	MTLS-USD-STK.SMART加载进度：###### [62%]
2021-02-28 16:58:56.530246	MTLS-USD-STK.SMART加载进度：###### [66%]
2021-02-28 16:58:56.533239	MTLS-USD-STK.SMART加载进度：###### [70%]
2021-02-28 16:58:56.538227	MTLS-USD-STK.SMART加载进度：####### [74%]
2021-02-28 16:58:56.542214	MTLS-USD-STK.SMART加载进度：####### [78%]
2021-02-28 16:58:56.547201	MTLS-USD-STK.SMART加载进度：######## [82%]
2021-02-28 16:58:56.550193	MTLS-USD-STK.SMART加载进度：######## [86%]
2021-02-28 16:58:56.554181	MTLS-USD-STK.SMART加载进度：######### [91%]
2021-02-28 16:58:56.557711	MTLS-USD-STK.SMART加载进度：######### [95%]
2021-02-28 16:58:56.560178	MTLS-USD-STK.SMART加载进度：######### [99%]
2021-02-28 16:58:56.560178	MTLS-USD-STK.SMART历史数

2021-02-28 16:58:57.075884	PD-USD-STK.SMART加载进度：##### [53%]
2021-02-28 16:58:57.079878	PD-USD-STK.SMART加载进度：##### [58%]
2021-02-28 16:58:57.085858	PD-USD-STK.SMART加载进度：###### [62%]
2021-02-28 16:58:57.090844	PD-USD-STK.SMART加载进度：###### [66%]
2021-02-28 16:58:57.095831	PD-USD-STK.SMART加载进度：###### [70%]
2021-02-28 16:58:57.098823	PD-USD-STK.SMART加载进度：####### [74%]
2021-02-28 16:58:57.100818	PD-USD-STK.SMART加载进度：####### [78%]
2021-02-28 16:58:57.103810	PD-USD-STK.SMART加载进度：######## [82%]
2021-02-28 16:58:57.109794	PD-USD-STK.SMART加载进度：######## [86%]
2021-02-28 16:58:57.113783	PD-USD-STK.SMART加载进度：######### [91%]
2021-02-28 16:58:57.116775	PD-USD-STK.SMART加载进度：######### [95%]
2021-02-28 16:58:57.118770	PD-USD-STK.SMART加载进度：######### [99%]
2021-02-28 16:58:57.118770	PD-USD-STK.SMART历史数据加载完成，数据量：420
2021-02-28 16:58:57.119768	PINS-USD-STK.SMART加载进度： [4%]
2021-02-28 16:58:57.120765	PINS-USD-STK.SMART加载进度： [8%]
2021-02-28 16:58:57.120765	PINS-USD-STK.SMART加载进度：# [12%]
2021-02-28 16:58:57.12176

2021-02-28 16:58:57.637945	SHOP-USD-STK.SMART加载进度：####### [78%]
2021-02-28 16:58:57.641936	SHOP-USD-STK.SMART加载进度：######## [82%]
2021-02-28 16:58:57.644928	SHOP-USD-STK.SMART加载进度：######## [86%]
2021-02-28 16:58:57.647920	SHOP-USD-STK.SMART加载进度：######### [91%]
2021-02-28 16:58:57.653905	SHOP-USD-STK.SMART加载进度：######### [95%]
2021-02-28 16:58:57.656895	SHOP-USD-STK.SMART加载进度：######### [99%]
2021-02-28 16:58:57.656895	SHOP-USD-STK.SMART历史数据加载完成，数据量：504
2021-02-28 16:58:57.659887	SNPS-USD-STK.SMART加载进度： [4%]
2021-02-28 16:58:57.661881	SNPS-USD-STK.SMART加载进度： [8%]
2021-02-28 16:58:57.664873	SNPS-USD-STK.SMART加载进度：# [12%]
2021-02-28 16:58:57.667865	SNPS-USD-STK.SMART加载进度：# [16%]
2021-02-28 16:58:57.670859	SNPS-USD-STK.SMART加载进度：## [21%]
2021-02-28 16:58:57.673849	SNPS-USD-STK.SMART加载进度：## [25%]
2021-02-28 16:58:57.675845	SNPS-USD-STK.SMART加载进度：## [29%]
2021-02-28 16:58:57.678836	SNPS-USD-STK.SMART加载进度：### [33%]
2021-02-28 16:58:57.680831	SNPS-USD-STK.SMART加载进度：### [37%]
2021-02-28 16:58:57.6

2021-02-28 16:58:58.245588	SYRS-USD-STK.SMART加载进度：## [21%]
2021-02-28 16:58:58.248581	SYRS-USD-STK.SMART加载进度：## [25%]
2021-02-28 16:58:58.251572	SYRS-USD-STK.SMART加载进度：## [29%]
2021-02-28 16:58:58.254565	SYRS-USD-STK.SMART加载进度：### [33%]
2021-02-28 16:58:58.258554	SYRS-USD-STK.SMART加载进度：### [37%]
2021-02-28 16:58:58.261545	SYRS-USD-STK.SMART加载进度：#### [41%]
2021-02-28 16:58:58.264538	SYRS-USD-STK.SMART加载进度：#### [45%]
2021-02-28 16:58:58.267531	SYRS-USD-STK.SMART加载进度：#### [49%]
2021-02-28 16:58:58.271521	SYRS-USD-STK.SMART加载进度：##### [53%]
2021-02-28 16:58:58.276506	SYRS-USD-STK.SMART加载进度：##### [58%]
2021-02-28 16:58:58.278500	SYRS-USD-STK.SMART加载进度：###### [62%]
2021-02-28 16:58:58.281493	SYRS-USD-STK.SMART加载进度：###### [66%]
2021-02-28 16:58:58.284487	SYRS-USD-STK.SMART加载进度：###### [70%]
2021-02-28 16:58:58.287476	SYRS-USD-STK.SMART加载进度：####### [74%]
2021-02-28 16:58:58.290469	SYRS-USD-STK.SMART加载进度：####### [78%]
2021-02-28 16:58:58.293464	SYRS-USD-STK.SMART加载进度：######## [82%]
2021-02-28 16:

2021-02-28 16:58:58.810936	TWLO-USD-STK.SMART加载进度：######### [95%]
2021-02-28 16:58:58.814924	TWLO-USD-STK.SMART加载进度：######### [99%]
2021-02-28 16:58:58.814924	TWLO-USD-STK.SMART历史数据加载完成，数据量：504
2021-02-28 16:58:58.818912	TWOU-USD-STK.SMART加载进度： [4%]
2021-02-28 16:58:58.822902	TWOU-USD-STK.SMART加载进度： [8%]
2021-02-28 16:58:58.826891	TWOU-USD-STK.SMART加载进度：# [12%]
2021-02-28 16:58:58.829883	TWOU-USD-STK.SMART加载进度：# [16%]
2021-02-28 16:58:58.836865	TWOU-USD-STK.SMART加载进度：## [21%]
2021-02-28 16:58:58.840853	TWOU-USD-STK.SMART加载进度：## [25%]
2021-02-28 16:58:58.842848	TWOU-USD-STK.SMART加载进度：## [29%]
2021-02-28 16:58:58.846838	TWOU-USD-STK.SMART加载进度：### [33%]
2021-02-28 16:58:58.849829	TWOU-USD-STK.SMART加载进度：### [37%]
2021-02-28 16:58:58.851824	TWOU-USD-STK.SMART加载进度：#### [41%]
2021-02-28 16:58:58.854816	TWOU-USD-STK.SMART加载进度：#### [45%]
2021-02-28 16:58:58.857808	TWOU-USD-STK.SMART加载进度：#### [49%]
2021-02-28 16:58:58.860800	TWOU-USD-STK.SMART加载进度：##### [53%]
2021-02-28 16:58:58.864800	TWOU-USD-

2021-02-28 16:58:59.372196	ZM-USD-STK.SMART加载进度：# [12%]
2021-02-28 16:58:59.375192	ZM-USD-STK.SMART加载进度：# [16%]
2021-02-28 16:58:59.379176	ZM-USD-STK.SMART加载进度：## [21%]
2021-02-28 16:58:59.383161	ZM-USD-STK.SMART加载进度：## [25%]
2021-02-28 16:58:59.388148	ZM-USD-STK.SMART加载进度：## [29%]
2021-02-28 16:58:59.391141	ZM-USD-STK.SMART加载进度：### [33%]
2021-02-28 16:58:59.394131	ZM-USD-STK.SMART加载进度：### [37%]
2021-02-28 16:58:59.396126	ZM-USD-STK.SMART加载进度：#### [41%]
2021-02-28 16:58:59.399118	ZM-USD-STK.SMART加载进度：#### [45%]
2021-02-28 16:58:59.402110	ZM-USD-STK.SMART加载进度：#### [49%]
2021-02-28 16:58:59.405104	ZM-USD-STK.SMART加载进度：##### [53%]
2021-02-28 16:58:59.408094	ZM-USD-STK.SMART加载进度：##### [58%]
2021-02-28 16:58:59.411086	ZM-USD-STK.SMART加载进度：###### [62%]
2021-02-28 16:58:59.415076	ZM-USD-STK.SMART加载进度：###### [66%]
2021-02-28 16:58:59.418067	ZM-USD-STK.SMART加载进度：###### [70%]
2021-02-28 16:58:59.420062	ZM-USD-STK.SMART加载进度：####### [74%]
2021-02-28 16:58:59.423054	ZM-USD-STK.SMART加载进度：####### [78

# 优化参数

In [16]:
from vnpy.app.cta_strategy.backtesting import OptimizationSetting

In [ ]:
op_setting = OptimizationSetting()
op_setting.add